In [1]:
import tensorflow as tf
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import pickle
import time
from xgboost import XGBClassifier
from xgboost import XGBRegressor
import sys
sys.path.append('/work/users/d/d/ddinh/aaco/src')
from load_dataset import load_adni_data
from sklearn.metrics import roc_auc_score, average_precision_score, precision_recall_curve
import tensorflow as tf


2024-12-03 13:29:29.030963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-03 13:29:29.055528: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-03 13:29:29.063797: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 13:29:29.083431: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-03 13:29:31.933092: W tensorflow/compiler/tf2

## Data

In [2]:
train = np.loadtxt('/work/users/d/d/ddinh/aaco/input_data/MelbournePedestrian/MelbournePedestrian_TRAIN.txt')
Xtrn = train[:, 1:]
Ytrn = train[:, 0]
valid = np.loadtxt('/work/users/d/d/ddinh/aaco/input_data/MelbournePedestrian/MelbournePedestrian_TEST.txt')
Xval = valid[:, 1:]
Yval = valid[:, 0]

Ytrn = np.eye(11)[Ytrn.astype(int)]
Yval = np.eye(11)[Yval.astype(int)]
Ytrn = Ytrn[:, 1:]
Yval = Yval[:, 1:]

num_ts = Xtrn.shape[1]

In [3]:
# load pt file 
import torch 
# path = '/work/users/d/d/ddinh/aaco/results/dataset_adnimlp_imputation_exclude_nan_mask_less_features.pt'
path = '/work/users/d/d/ddinh/aaco/results/dataset_pedestrian_xgb.pt'
data = torch.load(path)
x = data['X']
y = data['y']
mask = data['mask']
action = data['Action']

In [4]:
merge_action = []
count = 0
for i in range(action.shape[0]):
    if action[i, 0] == 1:
        merge_action.append(action[i])
        count += 1
    else:
        merge_action[count - 1] += action[i]
merge_action = np.array(merge_action)
merge_action = merge_action[:,:-1]

In [5]:
import tensorflow as tf
# model = tf.keras.models.load_model('/work/users/d/d/ddinh/aaco/models/mlp.keras')
model = XGBClassifier()
model.load_model('/work/users/d/d/ddinh/aaco/models/pedestrian.model')

In [6]:
valid.shape

(2439, 25)

In [7]:
merge_action.shape

(2439, 24)

In [20]:
# d = 24
# B = np.concatenate(
# [np.sum(np.random.permutation(np.eye(d))[:, :np.random.randint(6)], 1, keepdims=True) for _ in range(merge_action.shape[0])],
# 1)
# B = np.float32(B.T)
# # B = np.expand_dims(B, axis=-1)

# merge_action = B
# np.sum(merge_action, 1).mean()
merge_action = np.ones_like(merge_action)

In [21]:
Xvalmasked = np.concatenate((Xval*merge_action, merge_action), 1)
val_preds = model.predict_proba(Xvalmasked) 
print(np.mean(np.round(val_preds)==Yval))

0.9863058630586305


In [22]:
np.sum(merge_action, 1).mean()

24.0

In [ ]:
# 0.9863058630586305